In [4]:
import requests
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
import os
import pickle
import smtplib, ssl

In [13]:
df_test = pd.DataFrame() #creating Empty DataFrame

In [2]:
#Preparing email to automatically send notifications
from email.mime.text import MIMEText
import base64
from __future__ import print_function
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/gmail.send'] #Scopes that will be allowed by the sender (in this case to send messages from my testing email)

#Creating functions to create and send an email
def create_message(sender, to, subject, message_text):
  message = MIMEText(message_text)
  message['to'] = to 
  message['from'] = sender 
  message['subject'] = subject
  raw_message = base64.urlsafe_b64encode(message.as_string().encode("utf-8"))
  return {
    'raw': raw_message.decode("utf-8")
  }

def send_message(service, user_id, message):
  try:
    message = service.users().messages().send(userId=user_id, body=message).execute()
    print('Message Id: %s' % message['id'])
    return message
  except Exception as e:
    print('An error occurred: %s' % e)
    return None

kase_update = send_message(service=build('gmail', 'v1', credentials=Credentials.from_authorized_user_file('token.json', SCOPES)), 
user_id='sultan.python.tests@gmail.com', 
message=create_message('sultan.python.tests@gmail.com', 'sultann1997@gmail.com', 'Kase Update', 'Kase DataFrame has been updated'))



Message Id: 17a6076f81feb8c0


In [15]:
URL = 'https://kase.kz/ru/documents/marketvaluation/'

if requests.get(URL).status_code == 200:
    print('Connected')
else:
    print("Could not connect into server")

soup = BeautifulSoup(requests.get(URL).text, features='html.parser')
items = soup.find_all('div', {'id':'a2021'})

#functin to donwload and unzip files
def download_unzip(url, extract_to='.'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)


for div in soup.find_all('div', {'id':'a2021'}):
    for li in div.find_all('li'):
        a = li.find('a')
        #check if the file was already downloaded in the common directory
        if a['href'].replace(r'/files/market_valuation/ru/2021/', '') in os.listdir(r'C:\Users\Sultan\Downloads\Big Data downloads\Kase zips check2'):
            pass
        else:
            url_zip = 'https://kase.kz/' + a['href']
            #create directories for each zipfile to be extracted in
            current_dir = os.path.abspath(r'C:\Users\Sultan\Downloads\Big Data downloads\Kase zips check2'+'\\'+a['href'].replace(r'/files/market_valuation/ru/2021/', ''))
            try: os.makedirs(current_dir)
            except FileExistsError:
                pass
            download_unzip(url_zip, current_dir)
            #zipfile contains txt and xlsx files, we take xlsx files
            excel_file = [i for i in os.listdir(current_dir) if i.endswith('xlsx')]
            try: 
                temp_df = pd.read_excel(current_dir +'\\'+ excel_file[0], engine='openpyxl')
                #DataFrame automatically takes first row as headers, whereas our headers are on the third row
                headers = temp_df.iloc[1]
                temp_df = temp_df[3:]
                temp_df.columns = headers
                temp_df.drop(temp_df.columns[0], axis=1, inplace=True)
                temp_df['File'] = excel_file[0]
                df_test = df_test.append(temp_df)
                df_test.reset_index(drop=True, inplace=True)
                #sending email in case if the dataframe has been updated
                send_message(service=build('gmail', 'v1', 
                credentials=Credentials.from_authorized_user_file('token.json', SCOPES)), 
                user_id='sultan.python.tests@gmail.com', 
                message=create_message('sultan.python.tests@gmail.com', 'sultann1997@gmail.com', 'Kase Update', 'Kase DataFrame has been updated'))
            except (IndexError, AttributeError):
                pass


Connected
Message Id: 17a57281dc224a25
Message Id: 17a5728217862ece
Message Id: 17a5728263b33c54
Message Id: 17a57282e1fa1e16
Message Id: 17a57283464f6b71
Message Id: 17a57283b3261e98
Message Id: 17a572841c2399a3
Message Id: 17a57284ab2ba536
Message Id: 17a5728507c1526b
Message Id: 17a572857b6d2f43
Message Id: 17a57285bc7cbab6
Message Id: 17a5728611f8d87e
Message Id: 17a57286476acddb
Message Id: 17a57286bfae73f1
Message Id: 17a572871f57f0cf
Message Id: 17a572875268ccbb
Message Id: 17a57287b85ab340
Message Id: 17a5728806f5d347
Message Id: 17a5728845c25de5
Message Id: 17a57288ab751055
Message Id: 17a57288eccd90cd
Message Id: 17a5728950c70470
Message Id: 17a5728999eac58a
Message Id: 17a57289e89a1c0d
Message Id: 17a5728a5bde11aa
Message Id: 17a5728aa06fec11
Message Id: 17a5728ae17b8787
Message Id: 17a5728b58596919
Message Id: 17a5728ba3d883aa
Message Id: 17a5728bf3bfe3b5
Message Id: 17a5728c4283f417
Message Id: 17a5728c8203f3bd
Message Id: 17a5728cd21b1b5f
Message Id: 17a5728d34facb3d
Mess

In [18]:
df_test

1,Торговый код,ISIN,Вид ценной бумаги,Краткое наименование эмитента,Расчетная цена,Расчетная грязная,"Доходность до погашения, % годовых",Дней до погашения,Единица измерения цены,Тип цены,File
0,ABK024_063,KZMJ00000636,государственные ценные бумаги,Акимат Актюбинской области,100,"100,08460","0,35000",273,Чистая цена в%,Индикативная,Price29_06_2021.xlsx
1,ABK119_006,KZMF00000069,государственные ценные бумаги,Акимат Актюбинской области,100,"100,06780","6,09000",3206,Чистая цена в%,Индикативная,Price29_06_2021.xlsx
2,ABK131_021,KZMF00000218,государственные ценные бумаги,Акимат Актюбинской области,100,"105,33750","6,09000",3615,Чистая цена в%,Индикативная,Price29_06_2021.xlsx
3,ABK143_032,KZMF00000325,государственные ценные бумаги,Акимат Актюбинской области,100,"104,18530","6,09000",4043,Чистая цена в%,Индикативная,Price29_06_2021.xlsx
4,ABK143_033,KZMF00000333,государственные ценные бумаги,Акимат Актюбинской области,100,"104,18530","6,09000",4043,Чистая цена в%,Индикативная,Price29_06_2021.xlsx
...,...,...,...,...,...,...,...,...,...,...,...
81667,ZKK131_012,KZMF00000127,государственные ценные бумаги,Акимат Западно-Казахстанской области,100,"102,38920","6,08000",3789,Чистая цена в%,Индикативная,Price05_01_21(1).xlsx
81668,ZKK143_013,KZMF00000135,государственные ценные бумаги,Акимат Западно-Казахстанской области,100,"101,23690","6,09000",4217,Чистая цена в%,Индикативная,Price05_01_21(1).xlsx
81669,ZKK143_044,KZMF00000440,государственные ценные бумаги,Акимат Западно-Казахстанской области,100,"101,23690","6,09000",4217,Чистая цена в%,Индикативная,Price05_01_21(1).xlsx
81670,ZNGR,KZ000A1CTM83,акции,"АО ""Казахский Республиканский Торговый Дом ""ЗА...",0.01,NaN,NaN,NaN,Тенге,Индикативная,Price05_01_21(1).xlsx


In [17]:
#exporting file into excel
df_test.to_excel(r"C:\Users\Sultan\Downloads\Big Data downloads\my_excel.xls", engine='openpyxl')